In [1]:
%run 0.0_Dependance.ipynb

In [50]:
df = pd.read_csv("train.csv",index_col=0)
df.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
id,,,,,,,,,,,,,,,,,,,
0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [51]:
df["N_Week"]=np.round((df['N_Days']/7),1)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)
df.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,...,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,N_Week,N_Year
id,,,,,,,,,,,,,,,,,,,,,
0,999,D-penicillamine,59.0,M,N,N,N,N,2.3,316.0,...,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D,142.7,2.74
1,2574,Placebo,52.7,F,N,N,N,N,0.9,364.0,...,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C,367.7,7.05
2,3428,Placebo,37.6,F,N,Y,Y,Y,3.3,299.0,...,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D,489.7,9.39
3,2576,Placebo,50.6,F,N,N,N,N,0.6,256.0,...,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C,368.0,7.06
4,788,Placebo,45.6,F,N,Y,N,N,1.1,346.0,...,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C,112.6,2.16


In [52]:
#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

In [44]:
df.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,...,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,N_Week,N_Year
id,,,,,,,,,,,,,,,,,,,,,
0,999,0,59.0,1,0,0,0,0,2.3,316.0,...,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D,142.7,2.74
1,2574,1,52.7,0,0,0,0,0,0.9,364.0,...,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C,367.7,7.05
2,3428,1,37.6,0,0,1,1,2,3.3,299.0,...,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D,489.7,9.39
3,2576,1,50.6,0,0,0,0,0,0.6,256.0,...,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C,368.0,7.06
4,788,1,45.6,0,0,1,0,0,1.1,346.0,...,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C,112.6,2.16


# xgboost

In [ ]:
!pip install xgboost

In [77]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

target = df.Status
data = df.drop(columns=["Status","N_Days","N_Week"])

#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Modèle instancié
model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.1, objective="multi:softprob")

# Randomized
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [2, 4, 6],
    'learning_rate': [0.1, 0.01, 0.05]
}
search = RandomizedSearchCV(model, param_grid, cv=3, scoring='neg_log_loss')
search.fit(X_train, y_train)

# Meilleur modèle après Grid Search
best_model = search.best_estimator_

# Faire des prédictions
proba = best_model.predict_proba(X_test)
predictions= best_model.predict(X_test)
logloss = log_loss(y_test, proba)

# Évaluer le modèle
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print("Log Loss: ", logloss)


[[893   0  73]
 [ 32   5  18]
 [160   1 399]]
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       966
           1       0.83      0.09      0.16        55
           2       0.81      0.71      0.76       560

    accuracy                           0.82      1581
   macro avg       0.82      0.58      0.60      1581
weighted avg       0.82      0.82      0.81      1581

Log Loss:  0.45925325814109524


In [80]:
search.best_params_

{'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 0.01}